### Databricks Catalogs


![](/Volumes/workspace/default/bronze_images/OBJECT_MODE.png)

### Create Catalog & Schemas

In [0]:
%sql
-- create catalog
CREATE CATALOG IF NOT EXISTS ecommerce;

-- Set active catalog
USE CATALOG ecommerce;

-- Create schemas (Bronze / Silver / Gold)
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;

### Verify Catalog & Schemas

In [0]:
%sql
SHOW CATALOGS;

catalog
ecommerce
samples
system
workspace


### Create Volume

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS bronze_images
COMMENT 'Volume to store images for bronze layer';

In [0]:
%sql
SHOW VOLUMES;

database,volume_name
default,bronze_images


In [0]:
# Path to your local image
# local_path = ""
# Path to volume
# volume_path = ""
# Copy the image
#dbutils.fs.cp(local_path, volume_path)


### Register Delta Tables in Unity Catalog

#### Bronze Table

In [0]:
%sql
USE CATALOG ecommerce;
USE SCHEMA bronze; 

In [0]:
%sql
CREATE TABLE IF NOT EXISTS events
USING DELTA;

### Silver Table

In [0]:
%sql
USE CATALOG ecommerce;
USE SCHEMA silver; 

In [0]:
%sql
CREATE TABLE IF NOT EXISTS events
USING DELTA;

### Gold Table

In [0]:
%sql
USE CATALOG ecommerce;
USE SCHEMA gold; 

### Permissions

Unity Catalog provides fine-grained access control. Permissions are applied to catalogs, schemas, tables, views, and functions.

Types:

A. Catalog-Level Permissions

1. CREATE SCHEMA – Can create schemas inside the catalog
2. USAGE – Can access the catalog (required to see/use objects)
3. MANAGE – Can manage the catalog (grant/revoke permissions)

B. Schema-Level Permissions
1. CREATE – Can create tables/views in the schema
2. USAGE – Can reference objects in the schema
3. MANAGE – Can manage schema (grant/revoke privileges on schema or tables inside)

C. Table/View-Level Permissions
1. SELECT – Can read the table/view
2. INSERT – Can insert rows
3. UPDATE – Can update rows
4. DELETE – Can delete rows
5. TRUNCATE – Can truncate the table
6. REFERENCES – Can reference table in a foreign key

D. Function-Level Permissions
1. USAGE – Can use the function

Example 1: GRANT SELECT ON TABLE gold.products TO `analysts@company.com`;</br>

Explanation:
- Who: analysts@company.com
- Object: gold.products table
- Permission: SELECT → read-only access
- Purpose: Analysts can query the table or view but cannot modify it.

Example 2: GRANT ALL PRIVILEGES ON SCHEMA silver TO `engineers@company.com`;</br>

Explanation:

- Who: engineers@company.com
- Object: silver schema
- Permission: ALL PRIVILEGES → can create/drop tables, select/insert/update/delete, manage permissions
- Purpose: Engineers can do full data operations in Silver layer, which is typical in ETL pipelines.

In [0]:
%sql
GRANT SELECT ON TABLE gold.products TO `tbhavya054@gmail.com`;
GRANT ALL PRIVILEGES ON SCHEMA silver TO `tbhavya054@gmail.com`;

### Describe gold tables

In [0]:
%sql
DESCRIBE TABLE gold.products;

col_name,data_type,comment


### Create Gold Delta Table

In [0]:
%sql
-- Drop the existing table if it exists
USE CATALOG ecommerce;
USE SCHEMA gold;
DROP TABLE IF EXISTS products;

-- create the table with correct columns
CREATE TABLE products (
  product_id INT,
  product_name STRING,
  revenue INT,
  conversion_rate DOUBLE,
  purchases INT
)
USING DELTA;

### Insert Data into Gold Table

In [0]:
%sql
INSERT INTO products VALUES
(1, 'Laptop', 5000, 0.05, 15),
(2, 'Phone', 3000, 0.10, 8),
(3, 'Headphones', 1000, 0.12, 20);

num_affected_rows,num_inserted_rows
3,3


### Verify Queries

In [0]:
%sql
SHOW TABLES IN bronze;
SHOW TABLES IN silver;
SHOW TABLES IN gold;

SELECT * FROM gold.products;
SELECT * FROM gold.top_products;


product_name,revenue,conversion_rate
Laptop,5000,0.05
Headphones,1000,0.12


### Create Controlled View (Gold Layer Access)

In [0]:
%sql
CREATE OR REPLACE VIEW top_products AS
SELECT product_name, revenue, conversion_rate
FROM products
WHERE purchases > 10
ORDER BY revenue DESC
LIMIT 100;

In [0]:
%sql
SHOW TABLES IN bronze;

database,tableName,isTemporary
bronze,events,false


In [0]:
%sql
SHOW TABLES IN silver;

database,tableName,isTemporary
silver,events,false


In [0]:
%sql
SHOW TABLES IN gold;

database,tableName,isTemporary
gold,products,false
gold,top_products,false
